In [11]:
# from ntpath import join
# from posixpath import split
# from tkinter.tix import Tree
# from turtle import Turtle, left
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import csv

def heuristic_algorithm(file_path):
    #machine[5][1]==3代表是第3台機器做第6個job的第2階段
    #completion time 亦然
    
    # ID ,processing time, valid machine, due time, stage
    # cmp is the value to compare
    print("\n\n\n\n我是---------------------------------------分隔線")
    class Jobs:
        def __init__(self, ID, pt1, pt2, pm1, pm2, due, phase):
            self.ID = ID
            self.phase = phase #初始not allocated
            self.pt1 = pt1 #processing time
            self.pt2 = pt2
            self.pm1 = pm1 #can process machine
            self.pm2 = pm2
            self.due = due
            self.cmp = due-pt1-pt2 #緊急程度
    def test_all_jobs(jobs_list):
        for j in jobs_list:
            print("exact Job ID: ",j.ID,", Job.phase: ", j.phase, ", Jobs.cmp: ",j.cmp)
        print("end\n")
    jobs = []
    # read data and store the information into your self-defined variables
    fp = open(file_path, 'r', newline='')
    header = fp.readline()
    reader = csv.reader(fp, delimiter=',')
    machine_num = 0 #找出共有多少台機器在整個工廠
    for a_row in reader:
        #print(a_row)  # a_row is a list
        tmp = [item for item in a_row[3].split(',') if item != "N/A"]
        if(tmp):
            machine_num = int(max(tmp))
        tmp = [item for item in a_row[4].split(',') if item != "N/A"]
        if(tmp):
            machine_num = int(max(tmp))
        jobs.append(Jobs(int(a_row[0]), float(a_row[1]), float(a_row[2]),
                    a_row[3].split(','), a_row[4].split(','), float(a_row[5]), "Not allocated"))

    # start your algorithm here
    machine = []
    completion_time = []
    accu_machine = []
    stage_2 = []
    tardy = []
    for i in range(machine_num):
        accu_machine.append((i+1, 0)) #[(1,0), (2,0), ...(機器數,0)]
    #print(accu_machine)
    for i in range(len(jobs)):
        machine.append([0, 0])
        completion_time.append([0, 0])
    
    while(jobs):
        ptr = 0
        # sort joblists & machine accumulated time
        jobs = sorted(jobs, key=lambda job: (job.cmp, job.ID)) #先依照 cmp 再依照 ID
        accu_machine = sorted(
            accu_machine, key=lambda accu: (accu[1], accu[0])) #先依這台機器做最少的 再依照 ID
        job_now = jobs[0]
        # 如果出現了 cmp已經負數，那就沒可能做 直接丟tardy
        if(job_now.cmp < 0):
            print("job",job_now.ID,".cmp<0 ->tardy")
            tardy.append(jobs[0])
            jobs.pop(0)
            continue
        # 如果還沒分配過 分配stage 1 並把cmp 改成 due - pt2
        if(job_now.phase == "Not allocated"):
            while(str(accu_machine[ptr][0]) not in job_now.pm1 and ptr < len(accu_machine)):
                ptr += 1
            #print("test: ptr=",ptr,",who:", accu_machine[ptr][0],accu_machine,len(accu_machine))#找到可以塞的機器
            # 如果連stage 1在此刻就知道會爆 直接丟tardy 進入下一個while大迴圈（但理論上不會發生除非全爆
            # accu_machine[0] = (i,accumulated time in machine i)
            
            if(accu_machine[ptr][1]+job_now.pt1+job_now.pt2 > job_now.due): #最適合他的機器他都過不了=死去
                tardy.append(jobs[0])
                jobs.pop(0)
                continue
                
            #都沒事就繼續執行安穩塞進去  
            print("do normal :",job_now.ID,", stage1")
            machine[job_now.ID - 1][0] = accu_machine[ptr][0]
            completion_time[job_now.ID -
                            1][0] = accu_machine[ptr][1] + job_now.pt1
            # accu the time
            tmpL = list(accu_machine[ptr])
            tmpL[1] += job_now.pt1
            accu_machine[ptr] = tuple(tmpL)
            job_now.phase = "Stage 1"
            job_now.cmp = job_now.due - job_now.pt2
            #test_all_jobs(jobs)
            #print(accu_machine)
            # 檢查有沒有stage 2
            if('N/A' not in job_now.pm2):
                stage_2.append(job_now)
                jobs[0] = job_now
            else:
                completion_time[job_now.ID-1][1] = completion_time[job_now.ID-1][0] #若沒有stage2則完結時間同
                machine[job_now.ID-1][1] = machine[job_now.ID-1][0] #若沒有stage2則完結時間同
                jobs.pop(0)
        # 如果分配到stage1過了，那就分配stage2上去並把job丟掉 (完結他)
        # 基本上不會有塞不進去的可能，因為在後面的while迴圈可能一次就塞很多stage 2危險的進去了？ 再確認
        elif(job_now.phase == "Stage 1"):
            print("do normal :",job_now.ID,", stage2")
            while( ptr < len(accu_machine)-1):
                if(str(accu_machine[ptr][0]) not in job_now.pm2 or accu_machine[ptr][1]<=completion_time[job_now.ID-1][0]):
                    ptr += 1
                else:
                    break
            machine[job_now.ID - 1][1] = accu_machine[ptr][0]
            completion_time[job_now.ID -
                            1][1] = accu_machine[ptr][1] + job_now.pt2
            tmpL = list(accu_machine[ptr])
            tmpL[1] += job_now.pt2
            accu_machine[ptr] = tuple(tmpL)
            jobs.pop(0)
            # stage 2不再需要出現這個job
            for i, j in enumerate(stage_2):
                if j.ID == job_now.ID:
                    del stage_2[i]
                    break
        accu_machine = sorted(
            accu_machine, key=lambda accu: (accu[1], accu[0])) #先依這台機器做最少的 再依照 ID
        ptr = 0
        # To know
        emergency = True
        solved = False
        # sort stage2
        while(emergency and stage_2):
            emergency = False
            stage_2 = sorted(
                stage_2, key=lambda job: (job.cmp, job.pt2*-1))
            job2_now = stage_2[0]
            a = max(2, len(stage_2)/len(accu_machine))# make sure it can be done by the machine
            # 跑一遍有無任何機器觸發警告, 再由accu最小且能做此job的去做
            for k, (i, j) in enumerate(accu_machine):
                if(j + a*job2_now.pt2 >= job2_now.due):
                    emergency = True
                    solved = False
                    #print("emergency: ",job2_now.ID)
                    break
            # check 所有能做他的機器
            if(emergency == True and solved == False):
                for k, (i, j) in enumerate(accu_machine):
                    if(str(i) not in job2_now.pm2):
                        continue
                    if(job2_now.pt2 + j <= job2_now.due and accu_machine[k][1]>=completion_time[job2_now.ID-1][0]): #找到了accu最小能做此emergency job stage2的
                        print("emergency: ",job2_now.ID, "put in:",i)
                        machine[job2_now.ID-1][1] = i
                        completion_time[job2_now.ID -
                                        1][1] = j + job2_now.pt2
                        tmpL = list(accu_machine[k])
                        tmpL[1] += job2_now.pt2
                        accu_machine[k] = tuple(tmpL)
                        solved = True
                        accu_machine = sorted(
                            accu_machine, key=lambda accu: (accu[1], accu[0])) #先依這台機器做最少的 再依照 ID
                        break
            # if no machine can handle it, it's a tardy job
            if(emergency == True and solved == False):
                tardy.append(job2_now)
            if(emergency):
                stage_2.pop(0)
                for i, j in enumerate(jobs):
                    if j.ID == job2_now.ID:
                        del jobs[i]
                        break
            #emergency情況會繼續下去直到沒
    tardy_sum = len(tardy)
    print("全部結束後照processtime排")
    print("tardy: ")
    tardy = sorted(tardy, key=lambda job: job.pt1*-1)
    test_all_jobs(tardy) #此處演算法直接最長processtime塞最短機器,先塞完stage1
    for j in tardy:
        print(j.ID, j.phase)
    for i, j in enumerate(tardy):
        #print("(i,j.ID,j.phase)",i,j.ID, j.phase)
        if(j.phase == "Not allocated"):
            ptr = 0
            accu_machine = sorted(
                accu_machine, key=lambda accu: (accu[1], accu[0]))
            while(str(accu_machine[ptr][0]) not in j.pm1 and ptr < len(accu_machine)):
                ptr += 1
            #print(j.ID, j.pt1, "ptr:",ptr)
            #print("accu_machine:",accu_machine)
            machine[j.ID-1][0] = accu_machine[ptr][0]
            completion_time[j.ID-1][0] = accu_machine[ptr][1] + j.pt1
            tmpL = list(accu_machine[ptr])
            tmpL[1] += j.pt1
            accu_machine[ptr] = tuple(tmpL)
            # 檢查有沒有stage 2
            if('N/A' not in j.pm2): #有stage2
                print(j.ID,"has stage 2")
                j.phase = "Stage 1"
                j.cmp = j.due - j.pt2 #雖然不會用到了
            else: #沒stage2
                completion_time[j.ID-1][1] = completion_time[j.ID-1][0] #若沒有stage2則完結時間同
                machine[j.ID-1][1] = machine[j.ID-1][0] #若沒有stage2則完結時間同
                #del tardy[i]
                #print(i,tardy[i].ID)
                #del tardy[i]
                
    print("tardy after stage1: ")
    tardy = sorted(tardy, key=lambda job: job.pt2*-1) #再塞stage2
    test_all_jobs(tardy)
    for i,j in enumerate(tardy): 
        if(j.phase == "Stage 1"):
            ptr = 0
            accu_machine = sorted(
                accu_machine, key=lambda accu: (accu[1], accu[0]))
            while( ptr < len(accu_machine)-1 ):
                if((str(accu_machine[ptr][0]) not in j.pm2 or accu_machine[ptr][1]<=completion_time[j.ID-1][0])):
                    ptr += 1
                else:
                    break #找到可以的ptr了
            print(j.ID, j.pt1, "ptr:",ptr)
            #print("accu_machine:",accu_machine)
            machine[j.ID-1][1] = accu_machine[ptr][0]
            completion_time[j.ID-1][1] = accu_machine[ptr][1] + j.pt2
            tmpL = list(accu_machine[ptr])
            tmpL[1] += j.pt2
            accu_machine[ptr] = tuple(tmpL)
        #del tardy[i]
    return machine, completion_time, tardy_sum


for num in range(5):
    num_ = str(num + 1)
    i, j, k = heuristic_algorithm(f'to students/data/instance {num_}.csv')
    df = pd.read_csv(f'to students/data/instance {num_}.csv')
    print("instance ",num_)
    print(i)
    print(j)
    print("\n")
    ids, makespan= max(j, key=lambda item: item[1])
    print("tardy:", k)
    print("makespan: ",makespan)
#df





我是---------------------------------------分隔線
do normal : 1 , stage1
do normal : 7 , stage1
emergency:  7 put in: 2
do normal : 2 , stage1
emergency:  2 put in: 3
do normal : 3 , stage1
emergency:  3 put in: 4
do normal : 6 , stage1
do normal : 5 , stage1
emergency:  5 put in: 5
do normal : 4 , stage1
emergency:  4 put in: 4
do normal : 11 , stage1
do normal : 8 , stage1
do normal : 12 , stage1
do normal : 9 , stage1
do normal : 10 , stage1
do normal : 11 , stage2
do normal : 8 , stage2
do normal : 9 , stage2
do normal : 10 , stage2
全部結束後照processtime排
tardy: 
exact Job ID:  1 , Job.phase:  Stage 1 , Jobs.cmp:  3.7
end

1 Stage 1
tardy after stage1: 
exact Job ID:  1 , Job.phase:  Stage 1 , Jobs.cmp:  3.7
end

1 2.7 ptr: 0
instance  1
[[1, 4], [3, 3], [4, 4], [4, 4], [5, 5], [5, 5], [2, 2], [3, 5], [4, 3], [3, 2], [1, 1], [2, 2]]
[[2.7, 5.8999999999999995], [1.6, 3.0], [0.7, 2.5999999999999996], [3.0999999999999996, 3.8], [3.3, 4.3], [2.5, 2.5], [1.4, 3.4], [4.1, 5.4], [4.6, 5.8], [5